# Bonus Assignment 1

This is a Bonus Assignment try to complete all other Assignments before you reach here 

Try First Attempting Question 2 and then Question 1 


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Question 1
Solve this question on Kaggle! 
Download the notebook and submit it separately along with the other questions.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Use everything that you have learnt so far.

Analyse the data. Drop, scale or normalize features. Merge features if they correlated to make some meaningful features. 

Try out all the regression models. Use cross validation to find best parameters for each model.

# Question 2 : Regression using SVM and Tree Algorithms

In this question, we will be using the **insurance.csv** file which contain information on insurance charges based on the following informations: age,sex,bmi,region,number of children and whether the person is a smoker or not. You need to predict the charges based on the information given.

### 1. Load the data.

In [4]:
os.chdir(os.path.expanduser('~/Documents/projects/ML-fundamentals/Week 1/Bonus Assignment'))
insurance = pd.read_csv('insurance.csv')

In [5]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### 2. Separate the numerical and categorical columns.
### 3. Label Encode the categorical columns.
### 4. Scale the numerical columns. (Scale the charges separately so that you can calculate errors afterwards.)

In [13]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
categorical_features = ['sex', 'smoker', 'region']
lbl = LabelEncoder()
for feature in categorical_features:
    insurance[feature] = lbl.fit_transform(insurance[feature])

In [14]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [15]:
insurance.region.unique()

array([3, 2, 1, 0])

In [18]:
X = np.array(insurance.drop(columns=['charges'], axis=1))
y = np.array(insurance.charges)

In [19]:
scalerx = MinMaxScaler()
scalery = MinMaxScaler()
X = scalerx.fit_transform(X)
y = scalery.fit_transform(y.reshape(-1, 1))

In [21]:
y = y.reshape(1, -1)[0]
y

array([0.25161076, 0.00963595, 0.05311516, ..., 0.00810808, 0.01414352,
       0.44724873])

### 5. Split the data.

In [23]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

### 6. Support Vector Regressor

Here , you will use the SVR model from sklearn.svm and fit it on the training data. Then predict on the test data and calaculate MAE, MSE. But...

The SVR class contains many hyperparameters, example : kernel can have the following values : linear, rbf, poly, sigmoid.

Use **RandomizedSearchCV** from sklearn.model_selection , create a dictionary with keys 'kernel' and 'gamma' . As values of the keys, create a list of some possible values. Run a 3-fold cross validation test (cv=3) and find the best parameters. Then initiate the SVR model with those parameters.

In [25]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

In [41]:
hyp_para_dist = {
    'C' : range(1, 11),
    'kernel' : ['rbf', 'poly', 'linear', 'sigmoid'],
    'degree' : range(1, 5),
    'gamma' : np.linspace(0, 1, 10)
}
svr = SVR()

In [42]:
random_search = RandomizedSearchCV(estimator=svr, param_distributions=hyp_para_dist, random_state=42, scoring='accuracy')

In [45]:
random_search.fit(Xtrain, ytrain)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

RandomizedSearchCV(estimator=SVR(),
                   param_distributions={'C': range(1, 11),
                                        'degree': range(1, 5),
                                        'gamma': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
                                        'kernel': ['rbf', 'poly', 'linear',
                                                   'sigmoid']},
                   random_state=42, scoring='accuracy')

In [51]:
print("Best Parameters:", random_search.best_params_)

Best Parameters: {'kernel': 'linear', 'gamma': 0.1111111111111111, 'degree': 1, 'C': 8}


In [47]:
Svr = SVR(kernel='linear', gamma=0.1, degree=1, C=8)

In [48]:
Svr.fit(Xtrain, ytrain)

SVR(C=8, degree=1, gamma=0.1, kernel='linear')

In [49]:
ypred = Svr.predict(Xtest)

In [50]:
from sklearn.metrics import mean_squared_error

mean_squared_error(ytest, ypred)

0.009377275308037112